In [ ]:
import pandas as pd

In [ ]:
#load and concatenate all the daily parking tables
base = r'J:\Shared drives\TMD_TSA\Data\Parking\WebScraped_ParkingCost\scrape-parkme-main\html\\cp'
daily = []
monthly = []

for x in range(1, 35):
    try:
        table = pd.read_csv(base+str(x)+'.csv')
        daily.append(table)
    except:
        print(base+str(x)+'.csv')
    
    try:
        mable = pd.read_csv(base+'m'+str(x)+'.csv')
        monthly.append(mable)
    except:
        print(base+'m'+str(x)+'.csv')
    



In [ ]:
dpc = pd.concat(daily, ignore_index=True).drop_duplicates()
mpc = pd.concat(monthly, ignore_index=True).drop_duplicates()

In [ ]:
dpc

In [ ]:
dfil = dpc[(dpc['rate_simple'] != '—') & (~ pd.isna(dpc['daily_rates']))]
mfil = mpc[(mpc['rate_simple'] != '—') & (~ pd.isna(mpc['monthly_rates']))]
mfil

In [ ]:
dfil.to_csv(r'J:\Shared drives\TMD_TSA\Data\Parking\WebScraped_ParkingCost\scrape-parkme-main\daily_parking_cost.csv')
mfil.to_csv(r'J:\Shared drives\TMD_TSA\Data\Parking\WebScraped_ParkingCost\scrape-parkme-main\monthly_parking_cost.csv')

split the daily and monthly rate fields to calculate true daily and monthly rates

In [1]:
import geopandas as gpd
import fiona
import numpy as np
import pandas as pd

In [2]:
fiona.listlayers("J:/Shared drives/TMD_TSA/Data/Parking/WebScraped_ParkingCost/Geocoding_Parking_Lots.gdb")

['daily_parking_cost',
 'monthly_parking_cost',
 'monthly_parking_cost_Geocoded',
 'monthly_parking_cost_points',
 'daily_parking_cost_Geocoded',
 'parking_cost_dataset',
 'daily_parking_cost_Geocoded_unmatched',
 'parking_cost_dataset_trial',
 'lot_merged_full_Geocoded',
 'parking_cost_fullrec_NAP']

In [3]:
#want parking_cost_fullrec_NAP layer
lots=gpd.read_file("J:/Shared drives/TMD_TSA/Data/Parking/WebScraped_ParkingCost/Geocoding_Parking_Lots.gdb/", layer = 9)

In [4]:
lots['dr'] = lots['USER_daily_rates'].str.split(';', expand=False)
lots['dr']

0      [Each 30 Min: 6am-6pm $5,  Max: 6am-6pm $25,  ...
1      [Each 30 Min: 6am-6pm $5,  Max: 6am-6pm $25,  ...
2      [1 Hour:  $10,  2 Hours:  $15,  14 Hours:  $23...
3      [Each 30 Min: 6am-6pm $5,  Max: 6am-6pm $30,  ...
4      [30 Min:  $5,  1 Hour:  $10,  1.5 Hours:  $15,...
                             ...                        
462                              [Guests Only:  Free,  ]
463                              [Guests Only:  Free,  ]
464                              [Guests Only:  Free,  ]
465                     [6pm-8am:  Free,  Sun:  Free,  ]
466                                                 None
Name: dr, Length: 467, dtype: object

In [9]:
lots['max']=None
lots['free']=None

In [29]:
costDict={}
for inx, row in lots.iterrows():
    obama = []
    bernie = []
    if row['dr'] is not None:
        for x in range(0,len(row['dr'])):
            if "Max" in row['dr'][x]:
                obama.append(row['dr'][x])
            if "Total" in row['dr'][x]:
                obama.append(row['dr'][x])
            if "Day" in row['dr'][x]:
                obama.append(row['dr'][x])
            if "ree" in row['dr'][x]:
                bernie.append(row['dr'][x])
        costDict[row['IN_SingleLine']] = obama
        #row.loc['free'] = bernie
    if len(obama) < 1:
        costDict[row['IN_SingleLine']] = "No Max"

In [ ]:
sum(lots['max'].notna())

In [30]:
costDict

{'208 Friend St Boston, MA 02114, US ': [' Max: 6am-6pm $25'],
 '21 Lancaster St Boston, MA 02114, US ': [' Max: 6am-6pm $25'],
 '28 Ash St Boston, MA 02111, US ': 'No Max',
 '33 Essex St Boston, MA 02111, US ': [' Max: 6am-6pm $30'],
 '47 Lagrange St Boston, MA 02116, US ': [' Daily Max:  $25'],
 '78 Harrison Ave Boston, MA 02111, US ': [' Max: 6am-6pm $30',
  ' Max: 6pm-6am $25'],
 '290 Commercial Street Boston, MA 02109, US ': [' Daily Max:  $27'],
 '11 Stillings St Boston, MA 02210, US ': [' Daily Max:  $25',
  ' Max: With Validation $23'],
 '16 Charles St Boston, MA 02108, US ': [' Max: Mon-Fri',
  ' Max: Sat-Sun $18',
  ' Max: With Validation $16'],
 '40 Beach St Boston, MA 02111, US ': 'No Max',
 '98 Sudbury St Boston, MA 02108, US ': [' Daily Max:  $33',
  ' Max: Sat-Sun $16'],
 'Nassau St Boston, MA 02111, US ': 'No Max',
 '660 Washington St Boston, MA 02111, US ': 'No Max',
 '19 Lancaster St Boston, MA 02114, US ': [' Daily Max:  $25'],
 '274 Tremont St Boston, MA 02116, US '

In [ ]:
lots